In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
import cv2
import dlib
import numpy as np
import mysql.connector
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

# Load pre-trained models
facenet_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("Desktop/FYP/facial-landmarks-recognition-master/shape_predictor_68_face_landmarks.dat")

# Connect to MySQL database
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="shah1212",
    database="face_recognition_system"
)
cursor = db.cursor()

# Function to detect faces in an image
def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    return faces

# Function to recognize a face using VGGFace
def recognize_face(face_image):
    # Resize the face image to the required input size for VGGFace (224x224)
    face_image = cv2.resize(face_image, (224, 224))
    
    # Convert the image to float32 and normalize pixel values to [0, 1]
    face_image = face_image.astype('float32') / 255.0
    
    # Expand dimensions to match the input shape expected by VGGFace (batch size of 1)
    face_image = np.expand_dims(face_image, axis=0)
    
    # Preprocess the image for VGGFace
    face_image = preprocess_input(face_image, version=2)
    
    # Generate the face embedding using the VGGFace model
    embedding = facenet_model.predict(face_image)
    return embedding.flatten()  # Flatten the embedding to 1D array

# Function to compare face embedding with the database
def compare_with_database(embedding):
    # Fetch all face embeddings from the database
    cursor.execute("SELECT id, name, embedding FROM faces")
    faces = cursor.fetchall()
    
    # Initialize variables for the best match
    best_match_name = "Unknown"
    best_match_distance = float('inf')  # Initialize with a large value
    
    # Compare the input embedding with each embedding in the database
    for face in faces:
        db_embedding = np.frombuffer(face[2], dtype=np.float32)  # Convert BLOB to numpy array
        
        # Ensure the retrieved embedding has the same size as the input embedding
        if db_embedding.shape == embedding.shape:
            distance = np.linalg.norm(embedding - db_embedding)  # Calculate Euclidean distance
            
            # Update the best match if the current distance is smaller
            if distance < best_match_distance:
                best_match_distance = distance
                best_match_name = face[1]  # Name of the matched face
    
    # Set a threshold for recognition (adjust as needed)
    threshold = 0.6
    if best_match_distance < threshold:
        return best_match_name
    else:
        return "Unknown"

# Function to show database entries in a new window
def show_database():
    # Create a new window
    database_window = tk.Toplevel()
    database_window.title("Database Entries")
    database_window.geometry("600x400")

    # Create a treeview to display the database entries
    tree = ttk.Treeview(database_window, columns=("ID", "Name"), show="headings")
    tree.heading("ID", text="ID")
    tree.heading("Name", text="Name")
    tree.pack(fill="both", expand=True)

    # Fetch all faces from the database
    cursor.execute("SELECT id, name FROM faces")
    faces = cursor.fetchall()

    # Insert data into the treeview
    for face in faces:
        tree.insert("", "end", values=face)

# Main Dashboard Class
class MainDashboard:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Recognition System")
        self.root.geometry("800x600")

        # Live Feed Panel
        self.live_feed_label = tk.Label(root)
        self.live_feed_label.pack()

        # Recognition Results
        self.result_label = tk.Label(root, text="Recognition Results: None", font=("Arial", 14))
        self.result_label.pack()

        # Control Buttons
        self.start_button = tk.Button(root, text="Start Recognition", command=self.start_recognition)
        self.start_button.pack(side=tk.LEFT, padx=10)

        self.upload_button = tk.Button(root, text="Upload Image", command=self.upload_image)
        self.upload_button.pack(side=tk.LEFT, padx=10)

        self.database_button = tk.Button(root, text="View Database", command=show_database)
        self.database_button.pack(side=tk.LEFT, padx=10)

        # Start webcam feed
        self.cap = cv2.VideoCapture(0)
        self.update_frame()

    def start_recognition(self):
        ret, frame = self.cap.read()
        if ret:
            # Detect faces in the frame
            faces = detect_faces(frame)
            
            # Process each detected face
            for face in faces:
                x, y, w, h = face.left(), face.top(), face.width(), face.height()
                face_image = frame[y:y+h, x:x+w]
                
                # Recognize the face
                embedding = recognize_face(face_image)
                
                # Compare the embedding with the database
                recognized_name = compare_with_database(embedding)
                
                # Update the result label with the recognized name
                self.result_label.config(text=f"Recognition Results: {recognized_name}")
                
                # Draw a rectangle around the detected face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Display the frame with detected faces
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = ImageTk.PhotoImage(Image.fromarray(frame))
            self.live_feed_label.config(image=img)
            self.live_feed_label.image = img

    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            # Load the image using OpenCV
            image = cv2.imread(file_path)
            
            # Detect faces in the image
            faces = detect_faces(image)
            
            # Process each detected face
            for face in faces:
                x, y, w, h = face.left(), face.top(), face.width(), face.height()
                face_image = image[y:y+h, x:x+w]
                
                # Recognize the face
                embedding = recognize_face(face_image)
                
                # Compare the embedding with the database
                recognized_name = compare_with_database(embedding)
                
                # Update the result label with the recognized name
                self.result_label.config(text=f"Recognition Results: {recognized_name}")
                
                # Draw a rectangle around the detected face
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Display the image with detected faces
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            img = ImageTk.PhotoImage(Image.fromarray(image))
            self.live_feed_label.config(image=img)
            self.live_feed_label.image = img

    def update_frame(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = ImageTk.PhotoImage(Image.fromarray(frame))
            self.live_feed_label.config(image=img)
            self.live_feed_label.image = img
        self.root.after(10, self.update_frame)

# Create the Tkinter window
root = tk.Tk()
app = MainDashboard(root)
root.mainloop()

1/1 [==============================] - 3s 3s/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\shahz\anaconda3\envs\face_recognition_env\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\shahz\AppData\Local\Temp\ipykernel_12968\2143525356.py", line 176, in upload_image
    recognized_name = compare_with_database(embedding)
  File "C:\Users\shahz\AppData\Local\Temp\ipykernel_12968\2143525356.py", line 61, in compare_with_database
    db_embedding = np.frombuffer(face[2], dtype=np.float32)  # Convert BLOB to numpy array
ValueError: buffer size must be a multiple of element size


1/1 [==============================] - 0s 289ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\shahz\anaconda3\envs\face_recognition_env\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\shahz\AppData\Local\Temp\ipykernel_12968\2143525356.py", line 144, in start_recognition
    recognized_name = compare_with_database(embedding)
  File "C:\Users\shahz\AppData\Local\Temp\ipykernel_12968\2143525356.py", line 61, in compare_with_database
    db_embedding = np.frombuffer(face[2], dtype=np.float32)  # Convert BLOB to numpy array
ValueError: buffer size must be a multiple of element size


1/1 [==============================] - 0s 272ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\shahz\anaconda3\envs\face_recognition_env\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\shahz\AppData\Local\Temp\ipykernel_12968\2143525356.py", line 144, in start_recognition
    recognized_name = compare_with_database(embedding)
  File "C:\Users\shahz\AppData\Local\Temp\ipykernel_12968\2143525356.py", line 61, in compare_with_database
    db_embedding = np.frombuffer(face[2], dtype=np.float32)  # Convert BLOB to numpy array
ValueError: buffer size must be a multiple of element size
